In [1]:
%pip install selenium
%pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install selenium-wire

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 461.9 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install python-whois

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 325.8 kB/s eta 0:00:00 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103247 sha256=5fd184d026ba235f1119d1b4ea5e37566876049d112cb837f229e19f307ac3ff
  Stored in directory: /home/kali/.cache/pip/wheels/8a/d4/1d/bab4b44ad52eadf1b10c5c1ec7cb18a936f24b58bfb95b427e
Successfully built python-whois
Note: you may need to restart the kernel to use updated packages.


In [31]:
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import whois
import os

In [28]:
opts = Options()

# make web scraping 'invisible' if GUI is not required
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
url = "https://centralops.net/co/DomainDossier.aspx"
driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)
driver.get(url)
time.sleep(2)
search_box = driver.find_element(By.NAME, "addr")
search_box.send_keys("fmovies.to")
network_check_button = driver.find_element(By.NAME, "net_whois")
network_check_button.click()
search_button = driver.find_element(By.ID, "go")
search_button.click()
time.sleep(2)
raw_data = driver.find_element(By.TAG_NAME, 'pre')
print(raw_data)

/tmp/ipykernel_27584/3194533321.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"pre"}
  (Session info: chrome=113.0.5672.63)
Stacktrace:
#0 0x56106f10f133 <unknown>
#1 0x56106ee43966 <unknown>
#2 0x56106ee800dc <unknown>
#3 0x56106ee80211 <unknown>
#4 0x56106eeba6c4 <unknown>
#5 0x56106ee9fbfd <unknown>
#6 0x56106eeb81c7 <unknown>
#7 0x56106ee9f9a3 <unknown>
#8 0x56106ee7446a <unknown>
#9 0x56106ee7555e <unknown>
#10 0x56106f0cecae <unknown>
#11 0x56106f0d28fe <unknown>
#12 0x56106f0dbf20 <unknown>
#13 0x56106f0d3923 <unknown>
#14 0x56106f0a6c0e <unknown>
#15 0x56106f0f6b08 <unknown>
#16 0x56106f0f6c97 <unknown>
#17 0x56106f107113 <unknown>
#18 0x7f66bc49afd4 <unknown>


In [32]:
opts = Options()
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
url = "https://fmoviefree.sc"
directory = f"outputs/{url[8:]}"
os.makedirs(directory, exist_ok=True)
file_path = f"{directory}/whois.txt"
with open(file_path, "w") as file:
    w = whois.whois(url)
    file.write(f"{w.text}")
driver = webdriver.Chrome(options=opts)
driver.get(url)
home = driver.find_element(By.CSS_SELECTOR, "div.hm-button a")
home.click()
movie = driver.find_element(By.XPATH, "//div[@class='ml-item']/a[@title = 'Guardians of the Galaxy Vol. 3']")
movie_name = movie.get_attribute("title")
movie.click()
linking_site_url = driver.find_element(By.CSS_SELECTOR, "div#mv-info a")
linking_site_link = linking_site_url.get_attribute("href")
linking_site_url.click()
time.sleep(10)
file_path = f"{directory}/library_packages.txt"
with open(file_path, "w") as file:
    #print("linking site url: ", linking_site_link, file = file)
    file.write(f"linking site url: {linking_site_link}\n")
    hosting_site_url = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-embed").get_attribute("src")
    #print("hosting site url: ", hosting_site_url, file = file)
    file.write(f"hosting site url: {hosting_site_url}\n")
    script_tags = driver.find_element(By.TAG_NAME, "body").find_elements(By.CSS_SELECTOR, "script[src]")
    link_tags = driver.find_element(By.TAG_NAME, "head").find_elements(By.CSS_SELECTOR, "link[href]")
    script_head_tags = driver.find_element(By.TAG_NAME, "head").find_elements(By.CSS_SELECTOR, "script[src]")
    for script_tag in script_tags:
        src = script_tag.get_attribute("src")
        file.write(f"{src}\n")
    for link_tag in link_tags:
        href = link_tag.get_attribute("href")
        file.write(f"{href}\n")
    for script_head_tag in script_head_tags:
        src = script_head_tag.get_attribute("src")
        file.write(f"{src}\n")

In [29]:
url = "https://fmoviefree.sc"
w = whois.whois(url)
print(w.text)

Domain Name: FMOVIEFREE.SC
Registry Domain ID: D425500000340982222-AGRS
Registrar WHOIS Server:
Registrar URL: http://www.instra.com
Updated Date: 2023-03-19T20:47:53Z
Creation Date: 2023-03-19T20:32:07Z
Registry Expiry Date: 2024-03-19T20:32:07Z
Registrar Registration Expiration Date:
Registrar: Instra Corporation Pty Ltd.
Registrar IANA ID: 1376
Registrar Abuse Contact Email: abuse@instra.com
Registrar Abuse Contact Phone:
Reseller:
Domain Status: serverTransferProhibited https://icann.org/epp#serverTransferProhibited
Registrant Organization: Registrant of fmoviefree.sc
Registrant State/Province: Auckland District
Registrant Country: NZ
Name Server: LOLA.NS.CLOUDFLARE.COM
Name Server: YEVGEN.NS.CLOUDFLARE.COM
DNSSEC: unsigned

>>> Last update of WHOIS database: 2023-05-11T05:56:32Z <<<

For more information on Whois status codes, please visit https://icann.org/epp

Access to WHOIS information is provided to assist persons in determining the contents of a domain name registration reco